# 穴あきバケツのシミュレーション

制御技術を使うと、時間とともに状態が変化するシステムを、いい感じの状態に持って行ったり、あるいはいい感じの状態にキープしたりすることができます。
たとえばロボットの姿勢を操作して倒れないようにしたり、エアコンの出力を操作して部屋の温度を快適にしたりするために、制御は使われています。

制御の例を見るために、実機を毎回出してくるわけにはいかないので、シミュレーションを使いましょう。
ここではなるべく単純な例として、穴あきバケツの水位を制御することを考えます。

## 水槽のモデル

穴が開いているバケツを考えます。
簡単のためにバケツは円柱または角柱のような形をしているとします。
バケツの底に穴が開いていて、そこから水が漏れて行きます。
またバケツの上部には水を入れるためのバルブがあって、バルブの開き具合を操作することでバケツに水を補充することができるものとします。

<!-- この例における「バルブの開き具合」のような、システムに対する外部からの力や操作を **制御入力** と呼びます。-->

このとき、シミュレーションのためには以下のような情報を考えればよさそうです。

* バケツの断面積 $A$ [m²]
* 時刻 $t$ における水位 $h(t)$ [m]
* 時刻 $t$ においてバケツに注がれる水の量 $Q_{in} (t)$ [m³/s]
* 時刻 $t$ においてバケツから排出される水の量 $Q_{out} (t)$ [m³/s]

そうするとバケツの水位の変化量は以下の式を満たします。

$$
\frac{d h}{d t} (t) = \frac{ Q_{in}(t) - Q_{out} (t) }{ A }
$$

これはバケツの水位 $h$ に対する微分方程式になっています。
初期値を与えれば水位の時間変化をシミュレーションすることができそうですね。
初期値は $h(0) = 0$ としておきましょう。

しかし、まだ未確定のところがあります。
バケツに注がれる水の量 $Q_{in}$ は制御する側が適当に決めればいいですが、バケツから出ていく水の量 $Q_{out}$ はどうやって決まるのでしょうか。
物理の話になりますが、水位 $h(t)$ が高くなるほど排出圧力が増し、流出量が増加すると考えられますからそれを反映させましょう。
ここでは簡単のために、以下の式を満たすと仮定します。

$$
Q_{out} (t) = \sqrt{2 g h(t)}
$$

ただし $g$ は重力加速度 [m/s²] です。

これで情報がそろったので、バケツの数理モデルは完成ですね。

## 微分方程式を数値的に解く

常微分方程式を数値的に解く方法はいくつかあります。
オイラー法[^euler]とか、ルンゲクッタ法とか。
どちらも簡単に実装できるのですが、ここでは解き方は気にせず、ライブラリにお任せすることにします。
複数のライブラリがありますが、とりあえず SciPy の [`solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp) を使用します。
`solve_ivp` は、常微分方程式(ODE)の初期値問題(initial value problem)をいい感じに解いてくれる関数です。

簡単のために、つねに一定量（たとえば 10 [m³/s]）の水が流れ込んでくる想定としてシミュレーションをします。

[^euler]: オイラー法は簡単に実装できてコードが見やすいので例としてよく使われますが、精度が悪いので例としてしか使いません。


In [3]:
from dataclasses import dataclass
from typing import Callable
from math import sqrt
from scipy.integrate import solve_ivp
from scipy.integrate._ivp.ivp import OdeResult

GRAV : float = 9.8
"""重力加速度 [m/s²]"""

@dataclass
class Bucket:
    """バケツを表すクラス。バケツ固有の情報を保持する。"""

    A: float
    """バケツの断面積 [m²]"""

@dataclass
class BucketSystem:
    """制御対象としてのバケツを表すクラス"""

    backet: Bucket
    """バケツの情報"""

    # water_level: Callable[[float], float]
    # """各時刻における水位"""

    flow_in: Callable[[float], float]
    """各時刻における水の流入量"""

    # def flow_out(self, t: float) -> float:
    #     """各時刻における水の流出量"""
    #     return sqrt(2 * GRAV * self.water_level(t))

    def ode(self, t:float, h:float) -> float:
        """水位の微分方程式。水位の導関数を表す。"""
        return (self.flow_in(t) - sqrt(2 * GRAV * h)) / self.backet.A

def run_simulation() -> OdeResult:
    """シミュレーションを実行する関数。
    バケツの断面積や水の流入量はここで与えている。
    """

    # バケツの情報
    backet = Bucket(A=5.0)

    # バケツの動的な挙動を表すオブジェクト
    system = BucketSystem(
        backet=backet,
        flow_in=lambda t: 10.0
    )

    # 水位の初期値
    h0 = 0.0

    # シミュレーションの時間範囲
    t_span = (0, 20)

    # シミュレーションの実行
    sol = solve_ivp(
        fun=lambda t, h: system.ode(t, h),
        t_span=t_span,
        y0=[h0],
        max_step=0.01
    )

    return sol

run_simulation()

/tmp/ipykernel_3505/2109693963.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return (self.flow_in(t) - sqrt(2 * GRAV * h)) / self.backet.A


  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.000e-04 ...  1.999e+01  2.000e+01]
        y: [[ 0.000e+00  1.992e-04 ...  5.027e+00  5.027e+00]]
      sol: None
 t_events: None
 y_events: None
     nfev: 12020
     njev: 0
      nlu: 0